<a href="https://colab.research.google.com/github/viniciusds2020/ia_embedding_tensorflow_hub/blob/main/ia_embedding_tensorflow_hub.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Configuração do ambiente:

In [1]:
%%writefile requirements.txt
numpy
langchain_community
langchain_groq
langchain
python-dotenv
tensorflow
tensorflow_hub
pypdf
chromadb

Writing requirements.txt


In [ ]:
!pip install --quiet -r requirements.txt

In [3]:
import numpy as np
import tensorflow as tf
import tensorflow_hub as hub
from google.colab import userdata
from langchain_groq import ChatGroq
from langchain.document_loaders import PyPDFLoader
from langchain_community.vectorstores import Chroma
from langchain.chains import ConversationalRetrievalChain
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [4]:
# Bloco de codigo para VSCode:
# import os
#from dotenv import load_dotenv

# Carrega as variáveis do arquivo .env
#load_dotenv()

# Acesse as variáveis de ambiente
#api_key = os.getenv("GROQ_API_KEY")

### Configuração dos modelos e dos documentos para analise via llm

In [5]:
# Acesse as variáveis de ambiente
api_key = userdata.get('groqsecret')

In [6]:
# Carrega o PDF
loader = PyPDFLoader("/content/drive/MyDrive/geopy-readthedocs-io-en-latest.pdf")
pages = loader.load_and_split()

In [7]:
# Carrega o modelo de embeddings do TensorFlow Hub
embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder/4")

In [8]:
# Define uma função de embeddings para Chroma
class TensorFlowEmbeddingFunction:
    def __init__(self, model):
        self.model = model

    def embed_documents(self, texts):
        # Gera os embeddings para documentos
        embeddings = self.model(texts)
        return embeddings.numpy().tolist()  # Converte para uma lista de embeddings

    def embed_query(self, query):
        # Gera o embedding para a consulta
        query_embedding = self.model([query])
        return query_embedding.numpy().tolist()[0]  # Retorna como uma lista

### Armazenamento de vetores usando Chroma db

In [9]:
# Cria uma instância da função de embeddings
embedding_function = TensorFlowEmbeddingFunction(embed)

# Divide o texto em chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=2000, chunk_overlap=250)
texts = text_splitter.split_documents(pages)

# Cria o armazenamento de vetores usando Chroma
vectorstore = Chroma.from_documents(texts, embedding_function)

### Consulta de documentos

In [10]:
# Inicializa o modelo LLM
llm = ChatGroq(temperature=0.3, groq_api_key=api_key, model_name="mixtral-8x7b-32768")

In [11]:
pdf_qa = ConversationalRetrievalChain.from_llm(llm, vectorstore.as_retriever(), return_source_documents=True)
query = "É possivel utilizar o pandas dataframe? e responda em portugues do Brasil"
result = pdf_qa.invoke({"question": query, "chat_history": []})

In [12]:
result.keys()

dict_keys(['question', 'chat_history', 'answer', 'source_documents'])

In [15]:
display(result['answer'])

'Sim, é possível utilizar o DataFrame do pandas em conjunto com a biblioteca Geopy para geocodificar endereços e calcular distâncias entre pontos. No exemplo fornecido, é demonstrado como usar o DataFrame para armazenar os nomes das cidades e, em seguida, aplicar a função de geocodificação a cada nome de cidade para obter as localizações geográficas correspondentes. Em seguida, é calculado o ponto (latitude, longitude, altitude) para cada localização. Além disso, é possível customizar a função de geocodificação com opções adicionais, como o idioma ou os códigos de país. Por fim, é possível utilizar a classe RateLimiter para controlar a taxa de solicitações à API de geocodificação e evitar excessos de uso.'

In [16]:
display(result['source_documents'])

[Document(metadata={'page': 14, 'source': '/content/drive/MyDrive/geopy-readthedocs-io-en-latest.pdf'}, page_content='GeoPy Documentation, Release 2.4.1\nimport pandas as pd\ndf = pd.DataFrame({ \'name \': [\'paris \',\'berlin \',\'london \']})\nfrom geopy.geocoders import Nominatim\ngeolocator = Nominatim(user_agent="specify_your_app_name_here")\nfrom geopy.extra.rate_limiter import RateLimiter\ngeocode = RateLimiter(geolocator.geocode, min_delay_seconds=1)\ndf[\'location \'] = df[ \'name \'].apply(geocode)\ndf[\'point \'] = df[ \'location \'].apply( lambdaloc: tuple(loc.point) ifloc else None )\nThis would produce the following DataFrame:\n>>>df\nname location \\\n0 paris (Paris, Île-de-France, France métropolitaine, ...\n1 berlin (Berlin, 10117, Deutschland, (52.5170365, 13.3...\n2 london (London, Greater London, England, SW1A 2DU, UK...\npoint\n0 (48.8566101, 2.3514992, 0.0)\n1 (52.5170365, 13.3888599, 0.0)\n2 (51.5073219, -0.1276474, 0.0)\nTo pass extra options to the geocodecall: